In [4]:
# Import libraries
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json
from utils import process_runner_books
import pytz
import time

# Project libraries
import constants

# Change this certs path to wherever you're storing your certificates
certs_path = "/home/naing/certs/"
# Change these login details to your own
my_username = constants.USERNAME
my_password = constants.PASSWORD
# my_app_key = constants.API_KEY_DEMO #demo
my_app_key = constants.API_KEY_LIVE #live

trading = betfairlightweight.APIClient(username=my_username,
                                       password=my_password,
                                       app_key=my_app_key,
                                       certs=certs_path)

trading.login()

<LoginResource>

In [36]:
## TODO
debug = True # Print the data frames
liability_amount = 100

In [37]:
#######################################
# Filter out only greyhoud races
#######################################
gh_racing_id = constants.GH_RACING_ID #Greyhound racing ID
greyhound_racing_filter = filters.market_filter(
    event_type_ids=[gh_racing_id],
    market_countries=['GB'],
    market_type_codes=['FORECAST'],
    market_start_time={
        'to': (datetime.datetime.utcnow() + datetime.timedelta(hours=6)).strftime("%Y-%m-%dT%TZ") #sydtime 5pm to 7am next day for GB gh races
    }
)
#  This returns a list
gb_gh_events = trading.betting.list_events(
    filter=greyhound_racing_filter)

if debug:
    # Create a DataFrame with all the events by iterating over each event object
    gb_gh_events_df = pd.DataFrame({
        'Event Name': [event_object.event.name for event_object in gb_gh_events],
        'Event ID': [event_object.event.id for event_object in gb_gh_events],
        'Event Venue': [event_object.event.venue for event_object in gb_gh_events],
        'Country Code': [event_object.event.country_code for event_object in gb_gh_events],
        'Time Zone': [event_object.event.time_zone for event_object in gb_gh_events],
        'Open Date': [event_object.event.open_date for event_object in gb_gh_events],
        'Market Count': [event_object.market_count for event_object in gb_gh_events]
    })

    gb_gh_events_df

,Event Name,Event ID,Event Venue,Country Code,Time Zone,Open Date,Market Count
0,Monmore (F/C) 23rd Jun,31546305,Monmore,GB,Europe/London,2022-06-23 17:17:00,7
1,Sunderland (F/C) 23rd Jun,31546338,Sunderland,GB,Europe/London,2022-06-23 17:19:00,8
2,Newcastle (F/C) 23rd Jun,31546276,Newcastle,GB,Europe/London,2022-06-23 13:04:00,11
3,Perry Barr (F/C) 23rd Jun,31546404,Perry Barr,GB,Europe/London,2022-06-23 17:26:00,8
4,Swindon (F/C) 23rd Jun,31546361,Swindon,GB,Europe/London,2022-06-23 14:54:00,10
5,Hove (F/C) 23rd Jun,31546328,Hove,GB,Europe/London,2022-06-23 17:09:00,8
6,Romford (F/C) 23rd Jun,31546300,Romford,GB,Europe/London,2022-06-23 12:57:00,12
7,Crayford (F/C) 23rd Jun,31546286,Crayford,GB,Europe/London,2022-06-23 12:49:00,12


In [38]:
#######################################
#TODO: extract a list of IDs for the forecast markets
#######################################
fc_venue_ids = []
fc_venue_names = []
for eventObj in gb_gh_events:
    fc_venue_ids.append(eventObj.event.id)
    fc_venue_names.append(eventObj.event.name)

fc_venue_ids, fc_venue_names


(['31546305',
  '31546338',
  '31546276',
  '31546404',
  '31546361',
  '31546328',
  '31546300',
  '31546286'],
 ['Monmore (F/C) 23rd Jun',
  'Sunderland (F/C) 23rd Jun',
  'Newcastle (F/C) 23rd Jun',
  'Perry Barr (F/C) 23rd Jun',
  'Swindon (F/C) 23rd Jun',
  'Hove (F/C) 23rd Jun',
  'Romford (F/C) 23rd Jun',
  'Crayford (F/C) 23rd Jun'])

In [39]:
#######################################
## Filter out the next 5 upcoming games
#######################################
market_catalogue_filter = filters.market_filter(
    event_ids=fc_venue_ids)

market_catalogues = trading.betting.list_market_catalogue(
    filter=market_catalogue_filter,
    market_projection= ['MARKET_START_TIME', 'EVENT'],
    max_results='5',
    sort='FIRST_TO_START'
)

## show the dataframes if set
if debug:
    market_types_venueOfTheDay_df = pd.DataFrame({
        'Market Name': [market_cat_object.market_name for market_cat_object in market_catalogues],
        'Market ID': [market_cat_object.market_id for market_cat_object in market_catalogues],
        'Market Start Time': [market_cat_object.market_start_time for market_cat_object in market_catalogues],
        'Total Matched': [market_cat_object.total_matched for market_cat_object in market_catalogues],
        'Venue': [market_cat_object.event.venue for market_cat_object in market_catalogues]
        
    })

    market_types_venueOfTheDay_df

,Market Name,Market ID,Market Start Time,Total Matched,Venue
0,Forecast,1.200424339,2022-06-23 13:19:00,109.8764,Newcastle
1,Forecast,1.200424380,2022-06-23 13:23:00,7.0888,Crayford
2,Forecast,1.200424512,2022-06-23 13:32:00,0.0000,Romford
3,Forecast,1.200424340,2022-06-23 13:36:00,0.0000,Newcastle
4,Forecast,1.200424381,2022-06-23 13:42:00,0.0000,Crayford


In [40]:
#######################################
## Check the start time of the next game
## and sleep until 15 seconds before the game
#######################################
from utils import get_next_market

time_gap, myRaceID, myRaceVenue = get_next_market(market_catalogues)
# assert myRaceID == 0, "ERROR: myRaceID = 0" #TODO: remove in production

if(time_gap > datetime.timedelta(seconds=constants.PREBET_DELAY)):
    time_to_sleep = (time_gap - datetime.timedelta(seconds=constants.PREBET_DELAY)).seconds
    print("Sleeping for " + str(time_to_sleep) + " seconds")
    time.sleep(time_to_sleep)
    print("Sleeping done")
else:
    print("Don't need to sleep")



Time Now: 
2022-06-23 13:22:00
Found the market to lay: Name = crayford id = 1.200424380
Market Start Time: 2022-06-23 13:23:00+00:00
Sleeping for 39 seconds
Sleeping done


In [10]:
##TODO Choose the liability amount
# random from a list of 10 options
import numpy as np
import constants
liability_options = []

for i in range(18):
    if(len(liability_options) == 0):
        print("got here")
        liability_options = [20, 20, 20, 20, 20, 20, 20, 20, 100, 100] #not sure why this can't be moved into constants.py
    print("liability_options [BEFORE] = ", liability_options)

    liability_amount = np.random.choice(liability_options, size=1)
    print("Chosen Liability amount = ", liability_amount)

    liability_options.remove(liability_amount[0])
    print("liability_options [AFTER] = ", liability_options)

    print("list length = ", len(liability_options))

got here
liability_options [BEFORE] =  [20, 20, 20, 20, 20, 20, 20, 20, 100, 100]
Chosen Liability amount =  [20]
liability_options [AFTER] =  [20, 20, 20, 20, 20, 20, 20, 100, 100]
list length =  9
liability_options [BEFORE] =  [20, 20, 20, 20, 20, 20, 20, 100, 100]
Chosen Liability amount =  [20]
liability_options [AFTER] =  [20, 20, 20, 20, 20, 20, 100, 100]
list length =  8
liability_options [BEFORE] =  [20, 20, 20, 20, 20, 20, 100, 100]
Chosen Liability amount =  [100]
liability_options [AFTER] =  [20, 20, 20, 20, 20, 20, 100]
list length =  7
liability_options [BEFORE] =  [20, 20, 20, 20, 20, 20, 100]
Chosen Liability amount =  [20]
liability_options [AFTER] =  [20, 20, 20, 20, 20, 100]
list length =  6
liability_options [BEFORE] =  [20, 20, 20, 20, 20, 100]
Chosen Liability amount =  [20]
liability_options [AFTER] =  [20, 20, 20, 20, 100]
list length =  5
liability_options [BEFORE] =  [20, 20, 20, 20, 100]
Chosen Liability amount =  [100]
liability_options [AFTER] =  [20, 20, 20

In [44]:
#######################################
# Get a list of lay options
#######################################
# Create a price filter. Get all traded and offer data
price_filter = filters.price_projection(
    price_data=['EX_BEST_OFFERS']
)

# Request market books
market_books = trading.betting.list_market_book(
    market_ids=[myRaceID],
    price_projection=price_filter
)

# Grab the first market book from the returned list as we only requested one market 
market_book = market_books[0]
runners_df = process_runner_books(market_book.runners)

#TODO: extract a list of IDs for the lay options
lay_options_ids = []
for obj in market_book.runners:
    lay_options_ids.append(obj.selection_id)
# print(lay_options_ids)

#######################################
# Get odds from NEDS
#######################################
from utils import choose_lay_option

lay_selection_index = choose_lay_option(myRaceVenue)

if(lay_selection_index == -1):
    print("ERROR!!! 2 same odds found")
    assert()
    
elif(lay_selection_index == -2):
    print("ERROR!!! less than 6 runners")
    assert()

elif(lay_selection_index == -3):
    print("ERROR!!! NEDS API failed")
    assert()
else:
    print("Lay option found successfully")
    print(lay_selection_index)

lay_selection_id = lay_options_ids[lay_selection_index]
print("lay_selection_id = " + str(lay_selection_id))
fav_price = runners_df.loc[runners_df['Best Lay Price'].idxmin(), 'Best Lay Price']

#######################################
# Choose order type (limit order or market_on_close order - choose the latter)
#######################################
order_filter = filters.limit_order(
    price=fav_price,
    persistence_type='LAPSE',
    bet_target_type='PAYOUT',
    bet_target_size=liability_amount 
)
print(order_filter)

#######################################
# Create a place instruction filter
#######################################
instructions_filter = filters.place_instruction(
    selection_id = str(lay_selection_id),
    side="LAY",
    order_type = "LIMIT", # fixed price order
    limit_order=order_filter
)
print(instructions_filter)

True Venue Name = crayford
['4.50', '3.60', '5.50', '6.00', '3.80', '4.40']
['3.60', '3.80', '4.40', '4.50', '5.50', '6.00']
Forecast = 2-5
8
Lay option found successfully
8
lay_selection_id = 41419595
{'price': 14.0, 'persistenceType': 'LAPSE', 'betTargetType': 'PAYOUT', 'betTargetSize': 1200}
{'orderType': 'LIMIT', 'selectionId': '41419595', 'side': 'LAY', 'limitOrder': {'price': 14.0, 'persistenceType': 'LAPSE', 'betTargetType': 'PAYOUT', 'betTargetSize': 1200}}


In [45]:
#######################################
# Place the order
#######################################
order = trading.betting.place_orders(
    market_id = myRaceID,
    customer_strategy_ref='Naing_maker',
    instructions=[instructions_filter]
)

In [3]:
#######################################
# List the cleared orders
#######################################
myRaceID = 1.200450524
time.sleep(5)
current_orders = trading.betting.list_current_orders(
    customer_strategy_refs=['Naing_maker']).__dict__
# data = pd.DataFrame(current_orders['currentOrders']).head()
# print(current_orders)

cleared_orders = trading.betting.list_cleared_orders(
    bet_status="SETTLED",
    market_ids=[myRaceID])

# 
# Create a DataFrame from the orders
data = pd.DataFrame(cleared_orders._data['clearedOrders']).head()
print(data)

  eventTypeId   eventId     marketId  selectionId  handicap         betId  \
0        4339  31548500  1.200450524     41419593       0.0  272264823796   
1        4339  31548500  1.200450524     41419593       0.0  272264846627   
2        4339  31548500  1.200450524     41419593       0.0  272264865980   
3        4339  31548500  1.200450524     41419593       0.0  272264895029   

                 placedDate persistenceType orderType side betOutcome  \
0  2022-06-24T13:36:39.000Z           LAPSE     LIMIT  LAY       LOST   
1  2022-06-24T13:36:56.000Z           LAPSE     LIMIT  LAY       LOST   
2  2022-06-24T13:37:09.000Z           LAPSE     LIMIT  LAY       LOST   
3  2022-06-24T13:37:24.000Z           LAPSE     LIMIT  LAY       LOST   

   priceRequested               settledDate           lastMatchedDate  \
0            16.0  2022-06-24T13:42:43.000Z  2022-06-24T13:36:39.000Z   
1            18.0  2022-06-24T13:42:43.000Z  2022-06-24T13:36:58.000Z   
2            18.0  2022-06-24